## Load Coil20 data
## Extract DSIFT features

In [1]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.cd("./SSC_ADMM_v1.1")

'C:\\Users\\aleks_000\\Desktop\\Mentorship\\!GitHub'

In [2]:
from load import load_Coil20
from img2matrix import single_img2dsift

images_raw, labels = load_Coil20()
images_dsift = [single_img2dsift(image) for image in images_raw]


Loading Coil20...
----------------
Elapsed: 3.07 sec


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2)

ax[0].imshow(images_raw[0].reshape((-1, 128)));
ax[1].imshow(images_dsift[0].reshape((-1, 128)));

In [ ]:
import numpy as np

fig, ax = plt.subplots(1, 2)

ax[0].imshow(np.mean(images_raw, axis=0).reshape((-1, 128)));
ax[1].imshow(np.mean(images_dsift, axis=0).reshape((-1, 128)));

In [ ]:
fig, ax = plt.subplots(1, 2)
idx = 0

ax[0].imshow(np.mean(images_raw[72*idx:72*(idx+1)], axis=0).reshape((-1, 128)));
ax[1].imshow(np.mean(images_dsift[72*idx:72*(idx+1)], axis=0).reshape((-1, 128)));

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].imshow(np.std(images_raw, axis=0).reshape((-1, 128)));
ax[1].imshow(np.std(images_dsift, axis=0).reshape((-1, 128)));

In [6]:
from visualize import tSNE_2D, tSNE_3D

In [ ]:
images_raw_flat = images_raw.reshape(images_raw.shape[0], -1)
tSNE_2D(images_raw_flat, labels)
tSNE_3D(images_raw_flat, labels)

In [ ]:
tSNE_2D(images_dsift, labels)
tSNE_3D(images_dsift, labels)

## Perform PCA

In [7]:
from sklearn.decomposition import PCA
import numpy as np
    
pca = PCA(n_components=300, whiten=False, svd_solver='arpack', random_state=0)
#images_pca = pca.fit_transform(np.concatenate((1*images_raw, 10*images_dsift), axis=1))
images_pca = pca.fit_transform(images_dsift)

images_pca.shape

(1440, 300)

In [8]:
def display_image(image, height, width):
    print(np.min(image), np.max(image))
    imgplot = plt.imshow(image.reshape((height, width)))

In [ ]:
import matplotlib.pyplot as plt

# these are reduced parameters - not supposed to look like anything
display_image(images_pca[0], 10, 30)

In [ ]:
display_image(np.mean(images_pca, axis=0), 10, 30)

In [ ]:
display_image(np.std(images_pca, axis=0), 10, 30)

In [ ]:
tSNE_2D(images_pca, labels)
tSNE_3D(images_pca, labels)

## Normalize PCA output

In [12]:
# Types:
# feature-wise - normalization occurs along one pixel of all images
# image-wise - normalization occurs along all pixels of one image
# all - normalization occurs along all pixels of all images

# Methods:
# standard - mean is set to 0, std is set to 1
# [-1, 1] - min and max are used to linearly change the data range to [-1, 1]
# unit-vector - each bin is divided by its euclidean distance

#[BAD] feature, standard
#images_norm = (images_pca - np.mean(images_pca, axis=0)) / np.std(images_pca, axis=0)
# image, standard
#images_norm = (images_pca - np.mean(images_pca, axis=1)[:, np.newaxis]) / np.std(images_pca, axis=1)[:, np.newaxis]
# all, standard
#images_norm = (images_pca - np.mean(images_pca)) / np.std(images_pca)


#[BAD] feature, [-1, 1]
#mmin = np.min(images_pca, axis=0)
#mmax = np.max(images_pca, axis=0)
#[BAD] image, [-1, 1]
#mmin = np.min(images_pca, axis=1)[:, np.newaxis]
#mmax = np.max(images_pca, axis=1)[:, np.newaxis]
# all, [-1, 1]
mmin = np.min(images_pca)
mmax = np.max(images_pca)
# FOR ALL:
images_norm = (2*images_pca - mmax - mmin) / (mmax - mmin)

#[BAD] feature, unit
#images_norm = images_pca / np.sqrt(np.sum(images_pca*images_pca, axis=0))
# image, unit
#images_norm = images_pca / np.sqrt(np.sum(images_pca*images_pca, axis=1))[:, np.newaxis]
# all, unit
#images_norm = images_pca / np.sqrt(np.sum(images_pca*images_pca))

In [ ]:
import matplotlib.pyplot as plt

# these are reduced parameters - not supposed to look like anything
display_image(images_norm[0], 10, 30)

In [ ]:
display_image(np.mean(images_norm, axis=0), 10, 30)

In [ ]:
display_image(np.std(images_norm, axis=0), 10, 30)

In [ ]:
tSNE_2D(images_norm, labels)
tSNE_3D(images_norm, labels)

## Calculate C matrix

In [16]:
import numpy as np
import supporting_files.sda as sda

from supporting_files.helpers import optimize
from scipy.io import savemat, loadmat

In [334]:
# Matlab SSC #1
savemat('./temp.mat', mdict={'X': images_norm})
k = len(np.unique(labels))
alpha = 100.0
maxIter = 2
eng.SSC_modified(k, 0, False, alpha, False, 1, 1e-20, maxIter, False)
C = loadmat("./temp.mat")['C']

In [ ]:
display_image(C[:100, :100], 100, 100)
print(np.mean(np.square(C)))

In [ ]:
display_image(np.matmul(C, images_norm)[0], 10, 30)
print(np.mean(np.square(images_norm - np.matmul(C, images_norm))))

## Train Autoencoder

In [337]:
import dsc
import importlib
importlib.reload(dsc)

<module 'dsc' from 'C:\\Users\\aleks_000\\Desktop\\Mentorship\\!GitHub\\dsc.py'>

In [406]:
%%time
trainC = True
d = dsc.DeepSubspaceClustering(images_norm, C=C, trainC=trainC, hidden_dims=[200, 150, 200],
                               lambda1=1.0, lambda2=10.0, lambda3=100.0, learning_rate=0.003, weight_init='sda-normal',
                               weight_init_params=[1001, 0.006, images_norm.shape[0], 250],
                               optimizer='Adam', decay='sqrt', sda_optimizer='Adam', sda_decay='sqrt')

Layer 1
300 -> 200
epoch 0: global loss = 0.29537397623062134
epoch 250: global loss = 0.04538171738386154
epoch 500: global loss = 0.03850265219807625
epoch 750: global loss = 0.03508767858147621
epoch 1000: global loss = 0.03319395333528519
Layer 2
200 -> 150
epoch 0: global loss = 0.13996924459934235
epoch 250: global loss = 0.03345320373773575
epoch 500: global loss = 0.028261490166187286
epoch 750: global loss = 0.0266082976013422
epoch 1000: global loss = 0.02575034089386463
Wall time: 32.8 s


In [407]:
%%time
#d.optimizer = optimize(d.cost, 0.0003, 'Adam', 'sqrt', d.global_step)
d.train(batch_size=images_norm.shape[0], epochs=251, print_step=50)
images_HM2 = d.result
images_HM = d.reconstr
trained_C = np.float64(d.outC)


epoch 0: global loss = 0.921984851360321
epoch 50: global loss = 0.16839511692523956
epoch 100: global loss = 0.1511012315750122
epoch 150: global loss = 0.1394766867160797
epoch 200: global loss = 0.1300310641527176
epoch 250: global loss = 0.12194336205720901
Wall time: 41.4 s


In [ ]:
# lambda3 - regularization on trained_C
display_image(trained_C[:100, :100], 100, 100)

In [ ]:
display_image(images_HM2[0], 10, 15)

In [ ]:
# lambda1 - self-expressiveness
display_image(np.matmul(trained_C, images_HM2)[0], 10, 15)
print(np.mean(np.square(images_HM2 - np.matmul(trained_C, images_HM2))))

In [ ]:
display_image(np.mean(images_HM2, axis=0), 10, 15)

In [ ]:
display_image(np.std(images_HM2, axis=0), 10, 15)

In [245]:
%%time
#tSNE_2D(images_HM2, labels)
#tSNE_3D(images_HM2, labels)

Wall time: 0 ns


In [ ]:
# AE Reconstruction
fig, ax = plt.subplots(2)

index = 0;
ax[0].imshow(images_norm[index].reshape((10, 30)));
ax[1].imshow(images_HM[index].reshape((10, 30)));

In [383]:
# Matlab SSC #2
k = len(np.unique(labels))
alpha = 100.0
maxIter = 10
if(trainC):
    savemat('./temp.mat', mdict={'X': images_HM2})
else:
    savemat('./temp.mat', mdict={'C': trained_C})
grps = eng.SSC_modified(k, 0, False, alpha, False, 1, 1e-20, maxIter, True, 0, givenC)
C_after = loadmat("./temp.mat")['C']
labels_pred = np.asarray(grps, dtype=np.int32).flatten()

In [248]:
%%time
#tSNE_2D(images_HM2, labels_pred)
#tSNE_3D(images_HM2, labels_pred)

Wall time: 0 ns


## Perform clustering with SSC

In [384]:
from supporting_files.ji_zhang import err_rate
from sklearn.metrics import normalized_mutual_info_score as nmi
from sklearn.metrics import adjusted_rand_score as ari

print("Accuracy: ", str(1-err_rate(labels, labels_pred)))
print("NMI: ", str(nmi(labels, labels_pred, average_method="geometric")))
print("ARI: ", str(ari(labels, labels_pred)))

Accuracy:  0.36458333333333337
NMI:  0.597934898916298
ARI:  0.20502096981358237


In [116]:
# Python alternative:
from sklearn import cluster

def post_proC(C, K, d, alpha):
    # C: coefficient matrix, K: number of clusters, d: dimension of each subspace
    n = C.shape[0]
    C = 0.5*(C + C.T)
    C = C - np.diag(np.diag(C)) + np.eye(n,n) # for sparse C, this step will make the algorithm more numerically stable
    r = d*K + 1
    U, S, _ = svds(C,r,v0 = np.ones(n))
    U = U[:,::-1] 
    S = np.sqrt(S[::-1])
    S = np.diag(S)
    U = U.dot(S)
    U = normalize(U, norm='l2', axis = 1)  
    Z = U.dot(U.T)
    Z = Z * (Z>0)
    L = np.abs(Z ** alpha)
    L = L/L.max()
    L = 0.5 * (L + L.T)
    spectral = cluster.SpectralClustering(n_clusters=K, eigen_solver='arpack', affinity='precomputed', assign_labels='discretize')
    spectral.fit(L)
    grp = spectral.fit_predict(L) + 1
    return grp, L

def best_map(L1,L2):
    #L1 should be the labels and L2 should be the clustering number we got
    Label1 = np.unique(L1)
    nClass1 = len(Label1)
    Label2 = np.unique(L2)
    nClass2 = len(Label2)
    nClass = np.maximum(nClass1,nClass2)
    G = np.zeros((nClass,nClass))
    for i in range(nClass1):
        ind_cla1 = L1 == Label1[i]
        ind_cla1 = ind_cla1.astype(float)
        for j in range(nClass2):
            ind_cla2 = L2 == Label2[j]
            ind_cla2 = ind_cla2.astype(float)
            G[i,j] = np.sum(ind_cla2 * ind_cla1)
    m = Munkres()
    index = m.compute(-G.T)
    index = np.array(index)
    c = index[:,1]
    newL2 = np.zeros(L2.shape)
    for i in range(nClass2):
        newL2[L2 == Label2[i]] = Label1[c[i]]
    return newL2

In [ ]:
labels_pred2 = best_map(post_proC(C_after, k, 0, alpha)[0], labels)